In [1]:
import numpy as np
import pandas as pd
import trimesh as tm
from matplotlib import pyplot as plt
import torch as tc
from torch import Tensor, cdist

np.set_printoptions(precision=6, suppress=True)
tc.set_printoptions(precision=6)

In [2]:
def load_template(file_path):
        faces = []
        normals = []
        i = 0
        with open(file_path) as file:
            for line in file:
                if line[0] == "f":
                    line_split = line[1:].split()
                    line_split = np.array([i.split("//") for i in line_split])
                    face_temp = list(map(int, line_split[:, 0]))
                    faces.append(face_temp)
                    i += 1
        return np.array(faces), np.array(normals)

def load_mesh(file_path):
    vertex = []
    with open(file_path, "r") as f:
        for line in f:
            if line[0] == '#':
                continue
            elif "v " in line:
                line.replace('\n', ' ')
                tmp = list(map(float, line[1:].split()))
                vertex.append(tmp)
            else:
                break
    vertex = np.array(vertex)
    return vertex

def save_obj(path, pontos, faces=[]):
    with open(path, "w") as file:
        for ponto in pontos:
            file.write("v {} {} {}\n".format(ponto[0], ponto[1], ponto[2]))
        for face in faces:
            file.write("f {} {} {}\n".format(face[0], face[1], face[2]))

def generate_positions(vertices, coordinates):
    x = vertices[coordinates[:,0].to(tc.long)].T * coordinates[:,3]
    y = vertices[coordinates[:,1].to(tc.long)].T * coordinates[:,4]
    z = vertices[coordinates[:,2].to(tc.long)].T * coordinates[:,5]
    p = x+y+z
    return p.T

vertices = load_mesh('../curves-modules/mean_female_pose_0.obj')
faces, _ = load_template('../curves-modules/mean_female_pose_0.obj')
faces -= 1

In [87]:
def baricentric(a, b, c, p):
    
    vab = b - a
    vbc = c - b
    vca = a - c
    
    vap = p - a
    vbp = p - b
    vcp = p - c


    cross_norm = tc.linalg.norm(tc.cross(vab,vbc), axis=1)
    n = tc.cross(vab,vbc) / tc.column_stack([cross_norm, cross_norm, cross_norm])
    
    ABC = (n * tc.cross(vab, vbc)).sum(axis=1) / 2
    ABP = (n * tc.cross(vab, vbp)).sum(axis=1) / 2
    BCP = (n * tc.cross(vbc, vcp)).sum(axis=1) / 2
    CAP = (n * tc.cross(vca, vap)).sum(axis=1) / 2

    w = ABP/ABC
    u = BCP/ABC
    v = CAP/ABC

    colisions = tc.column_stack([u,v,w])
    return colisions

def intersection(p0, p1, p2, f1,f2,f3):

    vd = p0 - p1
    integer_part = tc.inner(p0, nplane)
    incognita_part = tc.inner(nplane, vd)
    t = -(dplane + integer_part)/incognita_part
    p = p0 + (tc.column_stack([t,t,t])*vd)
    return tc.column_stack([tc.column_stack([f1,f2,f3]), baricentric(p0,p1,p2,p)])


def calculate_colision(vertices, faces):
    global test
    v = vertices[faces]
    v = v.reshape(-1, 3)
    colision = (v@nplane) + dplane
    colision = colision.reshape(-1, 3)
    v = v.reshape(-1,3,3)

    t1 = tc.all(colision < 0, axis=1)
    t2 = tc.all(colision > 0, axis=1)
    t3 = ~(tc.any(tc.column_stack((t1,t2)), axis=1))

    f = faces[t3]
    colision = colision[t3]
    v = v[t3]
    
    t1 = colision[:,1] * colision[:,2] > 0
    t2 = colision[:,0] * colision[:,2] > 0
    t3 = colision[:,0] * colision[:,1] > 0

    p = tc.row_stack([
        intersection(v[t1][:,0], v[t1][:,1], v[t1][:,2], f[t1][:,0], f[t1][:,1], f[t1][:,2]),
        intersection(v[t1][:,0], v[t1][:,2], v[t1][:,1], f[t1][:,0], f[t1][:,2], f[t1][:,1]),

        intersection(v[t2][:,1], v[t2][:,2], v[t2][:,0], f[t2][:,1], f[t2][:,2], f[t2][:,0]),
        intersection(v[t2][:,1], v[t2][:,0], v[t2][:,2], f[t2][:,1], f[t2][:,0], f[t2][:,2]),

        intersection(v[t3][:,2], v[t3][:,0], v[t3][:,1], f[t3][:,2], f[t3][:,0], f[t3][:,1]),
        intersection(v[t3][:,2], v[t3][:,1], v[t3][:,0], f[t3][:,2], f[t3][:,1], f[t3][:,0])
    ])

    return p

def remove_mirror(positions, coordinates):
    absolute_positions = abs(positions)[:,0]
    min_point = absolute_positions.min(axis=0)[0]
    if min_point > 0.003:
        invalid_positions = positions[:,0] < 0
        coordinates = coordinates[invalid_positions]
    # absolute_positions = abs(positions)[:,1:3]
    # min_point = absolute_positions.min(axis=0)[0]
    # min_distance = tc.sqrt(tc.pow(min_point,2).sum())
    # if min_distance > 0.4:
    #     invalid_positions = positions[:,0] > 0
    #     coordinates = coordinates[invalid_positions]
    return coordinates


def sort_curve(positions, measure):
    centroid = positions.mean(axis=0)
    vectors = positions - centroid
    sort_around = vectors.var(axis=0).argmin()
    if sort_around == 0:
        angles = tc.arctan2(vectors[:,1], vectors[:,2])
    if sort_around == 1:
        angles = tc.arctan2(vectors[:,0], vectors[:,2])
    if sort_around == 2:
        angles = tc.arctan2(vectors[:,0], vectors[:,1])
    return measure[tc.argsort(angles)]

min_x = vertices[:,1].min()
max_x = vertices[:,1].max()

coordinates_array = []
test = []
for i in np.arange(min_x, max_x, 1e-3):
    nplane, dplane = tc.Tensor([0.0,1.0,0.0]), -i
    coordinates = calculate_colision(tc.Tensor(vertices), tc.from_numpy(faces))
    if coordinates.size:
        positions = generate_positions(tc.Tensor(vertices), coordinates)
        coordinates = remove_mirror(positions, coordinates)
        positions = generate_positions(tc.Tensor(vertices), coordinates)
        coordinates = sort_curve(positions, coordinates)
        coordinates_array.append(coordinates)
np.save('y.npy', np.array(coordinates_array))


/tmp/ipykernel_3239/551522158.py:111: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.save('y.npy', np.array(coordinates_array))
/tmp/ipykernel_3239/551522158.py:111: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save('y.npy', np.array(coordinates_array))


In [88]:
def calculate_distances(positions):
    if positions.numel() == 0:
        return 0
    positions = Tensor(positions)
    distances = cdist(positions, positions)
    distance = distances[0].sum() + distances[:,0][-1]
    return distance

measures_y = []
all_positions = []
for curve in np.load('y.npy', allow_pickle=True):
    positions = generate_positions(tc.Tensor(vertices), tc.Tensor(curve))
    all_positions.extend(positions)
    girth = calculate_distances(positions)
    measures_y.append(girth*100)
save_obj('test.obj', all_positions)

measures_x = []
for curve in np.load('x.npy', allow_pickle=True):
    positions = generate_positions(tc.Tensor(vertices), tc.Tensor(curve))
    girth = calculate_distances(positions)
    measures_x.append(girth*100)

In [89]:
life_data = pd.read_csv('../curves-modules/datasets/life.csv').dropna()
life_data[life_data['Gender']=='female']
life_data = life_data.drop(['Unnamed: 0', 'Body Type', 'Record Number', 'Age', 'Gender', 'BMI',
        'WHR', 'WHtR', 'ABSI', 'Age', 'DEV_WB_FROM_WAIST_F', 'DEV_WB_FROM_WAIST_B', 'DEV_WB_FROM_WAIST_S'], axis=1)

In [90]:
measures_y = tc.Tensor(measures_y).to(tc.float64)
data = tc.from_numpy(life_data.to_numpy()).to(tc.float64)
data_mean = data.mean(axis=0)

In [102]:
tc.set_printoptions(precision=6, sci_mode=False)
result = tc.abs(tc.unsqueeze(data_mean,1) - measures_y)
errors = (result.min(1)[0] / data_mean)
acepted = errors < 0.05
life_data.columns[acepted], errors[acepted]

(Index(['Height', 'Weight', 'NECK_HT', 'DIST_NECK_T_HIP', 'DIST_WAIST_KNEE',
        'CROTCH_HT', 'ARM_LTH_T_NECK_B_L', 'ARM_LTH_T_NECK_B_R',
        'ARM_LTH_T_NECK_R', 'INSEAM_L', 'INSEAM_R', 'HEAD_CIRC',
        'TOT_TORSO_GTH', 'SHOULDER_WTH_L', 'SHOULDER_WTH_R',
        'NECK_AC_BACK_WTH_AL'],
       dtype='object'),
 tensor([0.021673, 0.039000, 0.043702, 0.029442, 0.018008, 0.004300, 0.041034,
         0.046277, 0.044909, 0.011325, 0.015411, 0.029737, 0.020145, 0.042541,
         0.029376, 0.018269], dtype=torch.float64))